In [ ]:
# %% Deep learning - Section 13.14
#    Computation time

# This code pertains a deep learning course provided by Mike X. Cohen on Udemy:
#   > https://www.udemy.com/course/deeplearning_x
# The "base" code in this repository is adapted (with very minor modifications)
# from code developed by the course instructor (Mike X. Cohen), while the
# "exercises" and the "code challenges" contain more original solutions and
# creative input from my side. If you are interested in DL (and if you are
# reading this statement, chances are that you are), go check out the course, it
# is singularly good.


In [ ]:
# %% Two ways to track the time needed to fit models (exact and rough)


In [1]:
# %% Libraries and modules
import numpy               as np
import matplotlib.pyplot   as plt
import torch
import torch.nn            as nn
import seaborn             as sns
import copy
import torch.nn.functional as F
import pandas              as pd
import scipy.stats         as stats
import sklearn.metrics     as skm
import time

from torch.utils.data                 import DataLoader,TensorDataset
from sklearn.model_selection          import train_test_split
from google.colab                     import files
from torchsummary                     import summary
from IPython                          import display
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('svg')


In [7]:
# %% Function to get the data

# Load data
data_all = np.loadtxt(open('sample_data/mnist_train_small.csv','rb'),delimiter=',')

# Function
def get_dataset(digits=None,n_per_digits=None):

    # Remove labels (i.e., numbers IDs) from dataset
    labels = data_all[:,0]
    data   = data_all[:,1:]

    # Normalize to [0,1]
    data_norm = data / np.max(data)

    # Covert to tensor
    data_T   = torch.tensor(data_norm).float()
    labels_T = torch.tensor(labels).long()

    # Split data with scikitlearn
    train_data,test_data, train_labels,test_labels = train_test_split(data_T,labels_T,test_size=0.1)

    # Select subsample
    if digits is not None and n_per_digits is not None:

        keep_indices = np.zeros(labels.shape[0],dtype=bool)
        train_labels_np = train_labels.numpy()

        for lbl in np.unique(train_labels_np):

            idx = np.where(train_labels_np == lbl)[0]

            if lbl in digits:

                idx = np.random.choice(idx,n_per_digits,replace=False)

            keep_indices[idx] = True

        train_data   = train_data[keep_indices]
        train_labels = train_labels[keep_indices]

    # PyTorch datasets
    train_data = TensorDataset(train_data,train_labels)
    test_data  = TensorDataset(test_data,test_labels)

    # DataLoader objects
    batch_size   = 32
    train_loader = DataLoader(train_data,batch_size=batch_size,shuffle=True)
    test_loader  = DataLoader(test_data,batch_size=test_data.tensors[0].shape[0])

    return train_loader,test_loader


In [6]:
# %% Model class

def gen_model():

    class model(nn.Module):
        def __init__(self):
            super().__init__()

            # Architecture
            self.input  = nn.Linear(784,64 )
            self.hid1   = nn.Linear( 64,32 )
            self.hid2   = nn.Linear( 32,32 )
            self.output = nn.Linear( 32,10 )

        def forward(self,x):
            x = F.relu(self.input(x))
            x = F.relu(self.hid1(x))
            x = F.relu(self.hid2(x))

            return self.output(x)

    # Model instance, loss function, and optimizer
    ANN       = model()
    loss_fun  = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(ANN.parameters(),lr=0.01)

    return ANN,loss_fun,optimizer


In [13]:
# %% Function to train the model

def train_model():

    # Start timing (Option 1 - exact timing)
    timer_in_func = time.process_time()

    # Epochs and fresh model instance
    num_epochs = 10
    ANN,loss_fun,optimizer = gen_model()

    # Preallocate vars
    losses    = torch.zeros(num_epochs)
    train_acc = torch.zeros(num_epochs)
    test_acc  = torch.zeros(num_epochs)

    # Loop over epochs
    for epoch_i in range(num_epochs):

        # Loop over training data batches
        batch_acc  = []
        batch_loss = []

        for X,y in train_loader:

            # Forward pass, backpropagation, and optimizer step
            yHat = ANN(X)
            loss = loss_fun(yHat,y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Loss and accuracy from this batch
            batch_loss.append(loss.item())
            batch_acc.append( 100*torch.mean((torch.argmax(yHat,axis=1)==y).float()) )

        losses[epoch_i]    = np.mean(batch_loss).item()
        train_acc[epoch_i] = np.mean(batch_acc).item()

        # Test accuracy
        ANN.eval()

        with torch.no_grad():
            X,y = next(iter(test_loader))
            yHat = ANN(X)
            test_acc[epoch_i] = 100*torch.mean((torch.argmax(yHat,axis=1)==y).float())

        ANN.train()

        # Stop timing (print every 5h epoch)
        comp_time = time.process_time() - timer_in_func
        if (epoch_i + 1) % 5 == 0:
            print(f"Epoch {epoch_i+1}/{num_epochs}, elapsed time: {comp_time:.2f} s, test accuracy: {test_acc[-1]:.0f}%")

    return train_acc,test_acc,losses,ANN


In [ ]:
# %% Fit the model

train_loader,test_loader = get_dataset(digits=None,n_per_digits=None)
train_acc,test_acc,losses,ANN = train_model()


In [ ]:
# %% Fit the model again multiple times

# Start timing (Option 2 - rough timing)
timer_out_func = time.process_time()

for i in range(10):
    train_model()

exp_time = time.process_time() - timer_out_func
print(f"\n\nTotal elapsed experiment time: {exp_time/60:.0f} m")


In [ ]:
# %% Exercise 1
#    Modify the TotalExperimentTime code so that it prints minutes and seconds. For example, 500 seconds is
#    8 minutes and 20 seconds.

minutes = int(exp_time // 60)
seconds = int(exp_time % 60)

print(f"Total elapsed experiment time: {minutes} m {seconds} s")


In [ ]:
# %% Exercise 2
#    Modify the code inside the training function so that the display prints on only every 5th epoch.

# See modified function above
